# Tensorboard - XOR 사례
1. logging할 tensor를 결정
2. summary를 merge
3. writer 생성 및 그래프 추가
4. summary 실행 및 add_summary 실행
5. tensorboard 실행 후 브라우저에서 그래프 보기

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# 실행할 때마다 같은 결과를 출력하기 위한 seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

In [3]:
# x, y의 데이터 값
x_data = np.array([[0, 0],[0, 1],[1, 0],[1, 1]], dtype=np.float32) 
y_data = np.array([0, 1, 1, 0], dtype=np.float32).reshape(4, 1)

## 1. logging할 tensor를 결정

In [4]:
X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

In [5]:
# Layer 1
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="W1", dtype=tf.float32)
    b1 = tf.Variable(tf.random_normal([2]), name="b1", dtype=tf.float32)
    L1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)     # (2, 2)
    tf.summary.histogram("b1", b1)     # (2)
    tf.summary.histogram("Layer1", L1)

In [6]:
# Layer 2
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="W2", dtype=tf.float32)
    b2 = tf.Variable(tf.random_normal([1]), name="b2", dtype=tf.float32)
    hypo = tf.sigmoid(tf.matmul(L1, W2) + b2)

    tf.summary.histogram("W2", W2)     # (2, 1)
    tf.summary.histogram("b2", b2)     # (1)
    tf.summary.histogram("Hypothesis", hypo)

In [7]:
# Cost - Binary Cross Entropy
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypo) + (1.-Y) * tf.log(1.-hypo))
    tf.summary.scalar("Cost", cost)

In [8]:
# Optimizer - Adam
lr = 0.01
with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

In [9]:
# Accuracy 계산
predicted = tf.cast(hypo > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

## 2. summary를 merge
## 3. writer 생성 및 그래프 추가
## 4. summary 실행 및 add_summary 실행

In [10]:
# 학습
with tf.Session() as sess:
    # 2. Merge all summaries
    merged_summary = tf.summary.merge_all()
    # 3. Create writer and add graph
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    for step in range(3001):
        # 4. Run summary merge and add_summary
        _, summary, cost_val = sess.run([train, merged_summary, cost], 
                                        feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)
        if step % 1000 == 0:
            print(step, cost_val)
            
    # Accuracy report
    h, p, a = sess.run([hypo, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
        
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")

0 1.0279706
1000 0.13891393
2000 0.015897015
3000 0.0060498794

Hypothesis:
[[0.00447341]
 [0.9926696 ]
 [0.99263716]
 [0.00493696]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0
